In [11]:
import pandas as pd 
import numpy as np
from datetime import datetime

In [40]:
data = pd.read_csv("data/data_train/test_task_data.csv", sep=";")
data

,train_num,dep_st,ariv_st,price,dep_time,ariv_time
0,907,1909,1929,1790.87,00:50:00,20:00:00
1,908,1909,1929,1790.87,00:50:00,20:00:00
2,909,1929,1909,1790.87,10:01:00,01:00:00
3,909,1929,1921,199.15,11:10:00,01:00:00
4,910,1929,1909,1790.87,10:01:00,01:00:00
...,...,...,...,...,...,...
245,8501,1902,1937,195.00,05:50:00,21:50:00
246,8701,1902,1981,193.55,05:55:00,21:10:00
247,8801,1902,1981,193.40,06:35:00,19:30:00
248,8901,1937,1902,236.00,21:35:00,15:25:00


In [41]:
unique_stations = np.unique(data['ariv_st'])

unique_stations

array([1902, 1909, 1921, 1929, 1937, 1981], dtype=int64)

In [21]:
data.sort_values(["dep_st", "ariv_st", "price"])

,train_num,dep_st,ariv_st,price,dep_time,ariv_time
0,1,1,2,2,00:50:00,20:00:00
4,5,1,3,3,10:01:00,01:00:00
3,4,2,1,3,11:10:00,01:00:00
1,2,2,3,5,00:50:00,20:00:00
2,3,3,4,7,10:01:00,01:00:00


In [5]:
datetime_object = datetime.strptime(data["dep_time"][0], '%H:%M:%S')

In [22]:
def abs_value(a, b):
    diff = a - b
    return diff

data["dep_time"] = pd.to_timedelta(data["dep_time"])
data["ariv_time"] = pd.to_timedelta(data["ariv_time"])
data["time_diff"] = data.apply(lambda x: abs_value(x['ariv_time'], x['dep_time']), axis=1)
data_time = data.sort_values("time_diff")

In [42]:
d_price = {}
sorted_data = data.sort_values("price")
for item in unique_stations:
    d_price[item] = sorted_data[sorted_data["dep_st"] == item]
d_price

{1902:      train_num  dep_st  ariv_st   price  dep_time ariv_time
 58        1139    1902     1921  114.11  00:03:00  14:32:00
 10         919    1902     1981  130.60  04:59:00  20:06:00
 82        1181    1902     1937  131.34  04:50:00  15:52:00
 56        1137    1902     1937  131.34  19:53:00  08:21:00
 72        1163    1902     1981  131.84  04:37:00  15:52:00
 ..         ...     ...      ...     ...       ...       ...
 8          917    1902     1929  300.18  09:25:00  20:01:00
 217       2702    1902     1929  315.01  08:00:00  16:30:00
 218       2703    1902     1929  315.01  08:00:00  16:30:00
 216       2701    1902     1929  315.01  08:00:00  16:30:00
 213       2203    1902     1929  319.76  13:15:00  17:55:00
 
 [122 rows x 6 columns],
 1909:      train_num  dep_st  ariv_st    price  dep_time ariv_time
 77        1172    1909     1921  1407.37  12:17:00  15:47:00
 76        1171    1909     1921  1407.37  12:17:00  15:47:00
 1          908    1909     1929  1790.87  

In [43]:
unique_stations


array([1902, 1909, 1921, 1929, 1937, 1981], dtype=int64)

In [44]:
from time import sleep
passed_stations = [1]
path = []
paths = []
passed_train_iter = {}

def find_path(d_price, cur_station):
    while len(passed_stations) != len(unique_stations):
        print(passed_stations)
        possible_next_st = d_price[cur_station][~d_price[cur_station]['ariv_st'].isin(passed_stations)]
        print(possible_next_st)
        while possible_next_st.empty:
            print(f"Poped! {passed_stations.pop()}")
            if len(passed_stations) == 0:
                    break
            cur_station = passed_stations[-1]
            print(f'Current station: {cur_station}')
            possible_next_st = d_price[cur_station][~d_price[cur_station]['ariv_st'].isin(passed_stations)]
            print(possible_next_st)
            path.pop()
           
        if cur_station in passed_train_iter:
            print(f"len(possible_next_st), passed_train_iter[cur_station]: {len(possible_next_st)} and {passed_train_iter[cur_station]}")
            if len(possible_next_st) - 1 == passed_train_iter[cur_station]:
                pst = passed_stations.pop()
                passed_train_iter[pst] = 0
                print(f"Poped! {pst}")
                print(f"Dict: {passed_train_iter}")
                if len(passed_stations) == 0:
                    break
                cur_station = passed_stations[-1]
                print(f'Current station: {cur_station}')
                possible_next_st = d_price[cur_station][~d_price[cur_station]['ariv_st'].isin(passed_stations)]
                print(possible_next_st)
                path.pop()
            
#         print(d_price[cur_station])
        if cur_station not in passed_train_iter:
            passed_train_iter[cur_station] = 0
        else:
            passed_train_iter[cur_station] += 1
        
        print(f"Dict: {passed_train_iter}")
        if len(passed_stations) == 0:
                    break
        train = possible_next_st.iloc[passed_train_iter[cur_station]]
        print(f"Num: {train.train_num}")
        
        passed_stations.append(train.ariv_st)
        cur_station = train.ariv_st
        print(f'Current station: {cur_station}')
        path.append(train)
        
#         sleep(1)
for item in unique_stations:
    passed_stations = [item]
    path = []
    passed_train_iter = {}
    find_path(d_price, item)
    paths.append(path)

[1902]
     train_num  dep_st  ariv_st   price  dep_time ariv_time
58        1139    1902     1921  114.11  00:03:00  14:32:00
10         919    1902     1981  130.60  04:59:00  20:06:00
82        1181    1902     1937  131.34  04:50:00  15:52:00
56        1137    1902     1937  131.34  19:53:00  08:21:00
72        1163    1902     1981  131.84  04:37:00  15:52:00
..         ...     ...      ...     ...       ...       ...
8          917    1902     1929  300.18  09:25:00  20:01:00
217       2702    1902     1929  315.01  08:00:00  16:30:00
218       2703    1902     1929  315.01  08:00:00  16:30:00
216       2701    1902     1929  315.01  08:00:00  16:30:00
213       2203    1902     1929  319.76  13:15:00  17:55:00

[122 rows x 6 columns]
Dict: {1902: 0}
Num: 1139
Current station: 1921
[1902, 1921]
     train_num  dep_st  ariv_st    price  dep_time ariv_time
37        1068    1921     1929   130.36  06:45:00  21:22:00
30        1013    1921     1937   140.51  07:59:00  18:03:00
29   

123       1371    1921     1909  1421.12  05:35:00  07:30:00
len(possible_next_st), passed_train_iter[cur_station]: 6 and 5
Poped! 1921
Dict: {1929: 0, 1937: 0, 1902: 0, 1921: 0}
Current station: 1902
     train_num  dep_st  ariv_st   price  dep_time ariv_time
58        1139    1902     1921  114.11  00:03:00  14:32:00
10         919    1902     1981  130.60  04:59:00  20:06:00
72        1163    1902     1981  131.84  04:37:00  15:52:00
43        1105    1902     1981  138.54  00:14:00  15:22:00
207       2001    1902     1921  139.98  20:10:00  12:30:00
26         995    1902     1981  148.73  05:27:00  20:18:00
35        1066    1902     1981  152.58  03:47:00  18:37:00
25         987    1902     1981  156.90  02:52:00  18:49:00
249       9101    1902     1921  160.56  05:15:00  20:50:00
195       1812    1902     1921  161.35  21:35:00  14:25:00
199       1816    1902     1921  161.35  01:50:00  18:00:00
32        1026    1902     1921  164.63  06:46:00  22:23:00
48        1119    1

In [45]:
s = 0
i, j = 0, 0
for item in paths:
    j+=1
    s = 0
    for item2 in item:
        s += item2.price
        i += 1
        print (f"Num {i} for {j}: {item2.train_num}")
    print(f"Sum: {s}")
s, path

Num 1 for 1: 1139
Num 2 for 1: 1013
Num 3 for 1: 1155
Num 4 for 1: 1405
Num 5 for 1: 1367
Sum: 2516.59
Num 6 for 2: 1172
Num 7 for 2: 1140
Num 8 for 2: 919
Num 9 for 2: 1156
Num 10 for 2: 1148
Sum: 1927.1799999999998
Num 11 for 3: 1140
Num 12 for 3: 919
Num 13 for 3: 1156
Num 14 for 3: 1148
Num 15 for 3: 909
Sum: 2310.68
Num 16 for 4: 1146
Num 17 for 4: 1182
Num 18 for 4: 919
Num 19 for 4: 1126
Num 20 for 4: 1367
Sum: 2131.54
Num 21 for 5: 1182
Num 22 for 5: 1139
Num 23 for 5: 1125
Num 24 for 5: 1349
Num 25 for 5: 1367
Sum: 2513.21
Num 26 for 6: 920
Num 27 for 6: 1139
Num 28 for 6: 1013
Num 29 for 6: 1148
Num 30 for 6: 1367
Sum: 2300.1499999999996


(2300.1499999999996,
 [train_num         920
  dep_st           1981
  ariv_st          1902
  price          120.55
  dep_time     10:00:00
  ariv_time    01:05:00
  Name: 11, dtype: object,
  train_num        1139
  dep_st           1902
  ariv_st          1921
  price          114.11
  dep_time     00:03:00
  ariv_time    14:32:00
  Name: 58, dtype: object,
  train_num        1013
  dep_st           1921
  ariv_st          1937
  price          140.51
  dep_time     07:59:00
  ariv_time    18:03:00
  Name: 30, dtype: object,
  train_num        1148
  dep_st           1937
  ariv_st          1929
  price          134.11
  dep_time     09:40:00
  ariv_time    23:04:00
  Name: 67, dtype: object,
  train_num        1367
  dep_st           1929
  ariv_st          1909
  price         1790.87
  dep_time     11:21:00
  ariv_time    00:30:00
  Name: 119, dtype: object])